In [366]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random

In [367]:
# in starting points (13,0), the car should be able to also go to the another track
# in other words to (14,0), according to this grid this is the formost left lane

# in starting points (0,13), the car should be able to algo fo to the another track
# in other words (0,12) for example, accordint to this gris this is the foremost upway lane

# in starting points (12,24), the car should be able to also go to the another track
# in other words to (11,24), according to this grid this is the foremost right lane

#in starting points (24,14), the car should be able to also go to the another track
# in other words to (24,13), according to this grid this is the formost left lane

startingCoordinates = [(13,0),(0,11),(11,24),(24,13)]

movement1 = (0,1)
movement2 = (1,0)
movement3 = (0,-1)
movement4 = (-1,0)


In [368]:
class streetIntersection(ap.Model):

    def setup(self):

        #define the street limiter, that which will be black
        square = int((self.p.size-5)*2) #squares of size -5 as the resting number represents the street
        self.roadLimit_xy = ap.AgentList(self,square)
        self.roadLimit_x_y = ap.AgentList(self,square)
        self.roadLimitxy = ap.AgentList(self,square)
        self.roadLimitx_y = ap.AgentList(self,square)

        #define car agents
        self.carsl = ap.AgentList(self, self.p.n_cars)
        self.carsu = ap.AgentList(self, self.p.n_cars)
        self.carsr = ap.AgentList(self, self.p.n_cars)
        self.carsd = ap.AgentList(self, self.p.n_cars)

        self.singleCar = ap.AgentList(self,1)



        #Create street grid
        self.street = ap.Grid(self,[self.p.size]*2,track_empty=True)
        #add each delimiter
        carspos1 = [startingCoordinates[0] for i in range(len(self.carsl))]
        carspos2 = [startingCoordinates[1] for i in range(len(self.carsl))]
        carspos3 = [startingCoordinates[2] for i in range(len(self.carsl))]
        carspos4 = [startingCoordinates[3] for i in range(len(self.carsl))]

        self.street.add_agents(self.carsl,positions=carspos1)
        self.street.add_agents(self.carsu,positions=carspos2)
        self.street.add_agents(self.carsr,positions=carspos3)
        self.street.add_agents(self.carsd,positions=carspos4)


        #CONDITIONS -> 0:delimiters 1:cars 2:inactive
        self.roadLimit_xy.condition = 0
        self.carsl.condition = 2
        self.carsu.condition = 1
        self.carsr.condition = 3
        self.carsd.condition = 4


    
    def step(self):
        #we need to select and separate cars in different conditions as they move differently

        # left cars
        moving_carsl = self.carsl.select(self.carsl.condition == 2)
        # up cars
        moving_carsu = self.carsu.select(self.carsu.condition == 1)
        # right cars
        moving_carsr = self.carsr.select(self.carsr.condition == 3)
        # down cars
        moving_carsd = self.carsd.select(self.carsd.condition == 4)

        # left cars
        for car in moving_carsl:

            car.condition = 2

            current_pos = self.street.positions[car]
            move_byL = movement1[1]
            inFrontL = (current_pos[0], current_pos[1] + move_byL)

            if(inFrontL[1] < 25):

                print("current L: " + str(current_pos))
                print("front L: " + " " + str(inFrontL))

                if (self.street.agents[inFrontL].condition == 2):
                    print("crash avoid")
                    print("____")
                    continue
            
                self.street.move_by(car, movement1)
        
        # up cars
        for car2 in moving_carsu:

            car2.condition = 1

            current_pos = self.street.positions[car2]
            move_byU = movement2[0]
            inFrontU = (current_pos[0] + move_byU, current_pos[1])

            if(inFrontU[0] < 25):
                    
                    print("current U: " + str(current_pos))
                    print("front U: " + " " + str(inFrontU))
    
                    if (self.street.agents[inFrontU].condition == 1):
                        print("crash avoid")
                        print("____")
                        continue
                
                    self.street.move_by(car2, movement2)

        # right cars
        for car3 in moving_carsr:

            car3.condition = 3

            current_pos = self.street.positions[car3]
            move_byR = movement3[1]
            inFrontR = (current_pos[0], current_pos[1] + move_byR)

            if(inFrontR[1] > -1):
                    
                    print("current R: " + str(current_pos))
                    print("front R: " + " " + str(inFrontR))
    
                    if (self.street.agents[inFrontR].condition == 3):
                        print("crash avoid")
                        print("____")
                        continue
                
                    self.street.move_by(car3, movement3)
        
        # down cars
        for car4 in moving_carsd:

            car4.condition = 4

            current_pos = self.street.positions[car4]
            move_byD = movement4[0]
            inFrontD = (current_pos[0] + move_byD, current_pos[1])

            if(inFrontD[0] > -1):
                    
                    print("current D: " + str(current_pos))
                    print("front D: " + " " + str(inFrontD))
    
                    if (self.street.agents[inFrontD].condition == 4):
                        print("crash avoid")
                        print("____")
                        continue
                
                    self.street.move_by(car4, movement4)

        


In [369]:
parameters = {
    'size': 25,
    'steps': 18,
    'n_cars': 20,
}

In [370]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#000000', 1:'#0000FF',2:'#FFFFFF', 3:'#FF5733', 4:'#33FFC4', None: '#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"TESTING STREET\n")

fig, ax = plt.subplots()
model = streetIntersection(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=12))

current L: (13, 0)
front L:  (13, 1)
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash avoid
____
current L: (13, 0)
front L:  (13, 1)
crash av